In [9]:
pip install -U pulp

     |████████████████████████████████| 40.6 MB 12.9 MB/s eta 0:00:01     |██████████████████              | 23.0 MB 4.9 MB/s eta 0:00:04
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymysql
import re
import warnings
warnings.filterwarnings('ignore')

from pulp import *
import pandas as pd
import numpy as np

In [2]:
sql = '''SELECT *
 FROM recipe
'''

In [3]:
connection = pymysql.connect(
     user='root', 
        password='na12199611', 
        host='localhost', 
        database='recipe' ,
    charset='utf8')

try:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()

        new_result = [one for one in rows]
        data = pd.DataFrame(new_result)
        data.columns = ['recipe_id', 'recipe_name','category','preference', 'protein', 'fat', 'carbs', 'calorie', 'frequency','URL']

finally:
    connection.close()

In [297]:
n = len(data)

In [298]:
prob = pulp.LpProblem('menus', LpMinimize)
#data["x"] = [pulp.LpVariable("x_" + str(i), lowBound=0, upBound=7, cat=LpInteger) for i in range(n)]
data["x"] = [pulp.LpVariable("x_" + str(i), cat=LpBinary) for i in range(n)]

In [203]:
#prob = pulp.LpProblem('menus', LpMinimize)

In [204]:
#data['f'] = [pulp.LpVariable('f%d'%i, cat=LpBinary) for i in range(n)]

In [205]:
#prob += pulp.lpDot(data.calorie, data.x)

In [299]:
prob += pulp.lpDot(data.calorie, data.x)
prob += pulp.lpDot(data.frequency, data.x)

prob += pulp.lpDot(data.protein, data.x) <=48
prob += pulp.lpDot(data.protein, data.x) >=28
prob += pulp.lpDot(data.fat, data.x) <=29
prob += pulp.lpDot(data.fat, data.x) >=19
prob += pulp.lpDot(data.carbs, data.x) <=133
prob += pulp.lpDot(data.carbs, data.x) >=113
prob += pulp.lpDot(data.calorie, data.x) >=862

category = [1,2,3,4]
for c in list(set(data["category"]) ^ set(category)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == 0
for c, v in zip(category, (1, 2, 1,0)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == v

#for c, f in zip(category, (0, 2, 0,1)):
    #prob += pulp.lpSum(data.x[data["category"] == c]) == f
prob +=pulp.lpDot(data.preference, data.x) >= 13

In [142]:
prob += pulp.lpDot(data.calorie, data.x)
prob += pulp.lpDot(data.frequency, data.x)

prob += pulp.lpDot(data.protein, data.x) <=48
prob += pulp.lpDot(data.protein, data.x) >=28
prob += pulp.lpDot(data.fat, data.x) <=29
prob += pulp.lpDot(data.fat, data.x) >=19
prob += pulp.lpDot(data.carbs, data.x) <=133
prob += pulp.lpDot(data.carbs, data.x) >=113
prob += pulp.lpDot(data.calorie, data.x) >=862

category = [1,2,3,4]
for c in list(set(data["category"]) ^ set(category)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == 0
#for c, v in zip(category, (1, 2, 1,0)):
    #prob += pulp.lpSum(data.x[data["category"] == c]) == v

for c, f in zip(category, (0, 2, 0,1)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == f
prob +=pulp.lpDot(data.preference, data.x) >= 10

In [300]:
status = prob.solve()  # 解く
data['selector'] =0
if status == 1:  # 解けたら…
    data['selector'] = pd.Series([k.value() for k in data.x])  # df.x.value()で値の取得をしてdf.selectorに入れる

data_selected = data[data["selector"] >= 1]  # 選んだメニューだけのdfを作成
data_selected = data_selected.drop([ "x", "selector"], axis=1) # categoryとxを除外して表示

sum_row = {col:data_selected[col].sum() for col in data_selected}
sum_df = pd.DataFrame(sum_row, index=["Total"])
df = data_selected.append(sum_df)
df

TypeError: can only concatenate str (not "int") to str

In [144]:
data_selected

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie,frequency


In [132]:
t =(list(data.query('recipe_id == 81').index))

In [133]:
data_selected = data_selected.drop(t)
data_selected_list = data_selected.index.to_list()
data_selected_list

[6, 13, 18]

In [134]:
data_selected_list = data_selected.index.to_list()
data_selected_list

[6, 13, 18]

In [135]:
#data = data.drop(index = i)
#data = data.reset_index(drop = True)
for i in data_selected_list :
    data.iloc[i,8] += 1


In [83]:
#pd.set_option('display.max_rows', None)
data.iloc[14,8]

0

In [1969]:
print(154*0.8)
print(48*0.8)
print(30*0.8)
print(1078*0.8)

123.2
38.400000000000006
24.0
862.4000000000001


In [4]:
n = len(data)
prob = pulp.LpProblem('menus', LpMinimize)

data["x"] = [pulp.LpVariable("x_" + str(i), cat=LpBinary) for i in range(n)]

prob += pulp.lpDot(data.calorie, data.x)+100*pulp.lpDot(data.frequency, data.x)


prob += pulp.lpDot(data.protein, data.x) <=48
prob += pulp.lpDot(data.protein, data.x) >=28
prob += pulp.lpDot(data.fat, data.x) <=29
prob += pulp.lpDot(data.fat, data.x) >=19
prob += pulp.lpDot(data.carbs, data.x) <=133
prob += pulp.lpDot(data.carbs, data.x) >=113
prob += pulp.lpDot(data.calorie, data.x) >=862

category = [1,2,3,4]
for c in list(set(data["category"]) ^ set(category)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == 0
for c, v in zip(category, (1, 2, 1,0)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == v
#for c, f in zip(category, (0, 2, 0,1)):
    #prob += pulp.lpSum(data.x[data["category"] == c]) == f

prob +=pulp.lpDot(data.preference, data.x) >= 13
#prob +=pulp.lpDot(data.preference, data.x) >= 10

status = prob.solve()
data['selector'] =0
if status == 1:  
    data['selector'] = pd.Series([k.value() for k in data.x])  

data_selected = data[data["selector"] >= 1]  
data_selected = data_selected.drop([ "x", "selector",'URL'], axis=1) 

sum_row = {col:data_selected[col].sum() for col in data_selected}
sum_df = pd.DataFrame(sum_row, index=["Total"])
df = data_selected.append(sum_df)

t =(list(data.query('recipe_id == 81').index))

data_selected = data_selected.drop(t)
data_selected_list = data_selected.index.to_list()
for i in data_selected_list :
    data.iloc[i,8] += 1

df

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie,frequency
11,12,ピーマンと白滝の甘辛炒め,2,3,7,11,16,185,0
80,81,ごはん,3,3,4,1,56,234,0
90,91,四川風鶏むね肉のよだれ鶏,1,5,24,9,7,218,0
138,139,たっぷり野菜のチャプチェ,2,4,4,4,44,225,0
Total,323,ピーマンと白滝の甘辛炒めごはん四川風鶏むね肉のよだれ鶏たっぷり野菜のチャプチェ,8,15,39,25,123,862,0


In [365]:
n = len(data)
prob = pulp.LpProblem('menus', LpMinimize)
#data["x"] = [pulp.LpVariable("x_" + str(i), lowBound=0, upBound=7, cat=LpInteger) for i in range(n)]
data["x"] = [pulp.LpVariable("x_" + str(i), cat=LpBinary) for i in range(n)]
prob += pulp.lpDot(data.calorie, data.x)+ 100*pulp.lpDot(data.frequency, data.x)
#prob += pulp.lpDot(data.frequency, data.x)

prob += pulp.lpDot(data.protein, data.x) <=48
prob += pulp.lpDot(data.protein, data.x) >=28
prob += pulp.lpDot(data.fat, data.x) <=29
prob += pulp.lpDot(data.fat, data.x) >=19
prob += pulp.lpDot(data.carbs, data.x) <=133
prob += pulp.lpDot(data.carbs, data.x) >=113
prob += pulp.lpDot(data.calorie, data.x) >=862

category = [1,2,3,4]
for c in list(set(data["category"]) ^ set(category)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == 0
#for c, v in zip(category, (1, 2, 1,0)):
    #prob += pulp.lpSum(data.x[data["category"] == c]) == v

for c, f in zip(category, (0, 2, 0,1)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == f
prob +=pulp.lpDot(data.preference, data.x) >= 10

status = prob.solve()  # 解く
data['selector'] =0
if status == 1:  # 解けたら…
    data['selector'] = pd.Series([k.value() for k in data.x])  # df.x.value()で値の取得をしてdf.selectorに入れる

data_selected = data[data["selector"] >= 1]  # 選んだメニューだけのdfを作成
data_selected = data_selected.drop([ "x", "selector",'URL'], axis=1) # categoryとxを除外して表示

sum_row = {col:data_selected[col].sum() for col in data_selected}
sum_df = pd.DataFrame(sum_row, index=["Total"])
df = data_selected.append(sum_df)

data_selected_list = data_selected.index.to_list()
for i in data_selected_list :
    data.iloc[i,8] += 1
df

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie,frequency
59,60,かぶおろしのなめこ和え,2,1,3,0,13,60,0
106,107,簡単冷副菜！オクラとトマトの煮浸し,2,4,1,0,4,18,0
122,123,青唐辛子のドライカレー,4,5,25,22,102,790,0
Total,290,かぶおろしのなめこ和え簡単冷副菜！オクラとトマトの煮浸し青唐辛子のドライカレー,8,10,29,22,119,868,0


In [384]:
df.iloc[4,2] ='-'

In [385]:
df

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie,frequency
3,4,イワシの蒲焼き,1,4,41,18,26,469,0
19,20,さっぱり大根の甘酢漬け,2,3,1,1,27,114,0
80,81,ごはん,3,3,4,1,56,234,0
116,117,きゅうりと大根の花椒漬け,2,3,1,0,10,49,0
Total,-,-,-,13,47,20,119,866,0


In [388]:
data.iloc[116,9]

'https://www.kurashiru.com/recipes/1cb19203-10fc-4502-827e-82b54389f26c'